# Install Package

In [1]:
import pandas as pd
import numpy as np
import re

# Remove Stopword

In [2]:
from nltk.corpus import stopwords

def clean_stopword(text):
    # Apply this code to every textual string
    word_list = text.split() 
    filtered_words = [word for word in word_list if word not in stopwords.words('english')]
    text = ' '.join(filtered_words)
    return text

# Remove Punctuation

In [3]:
import string
print(string.punctuation)

def remove_punc(text):
    punctuation = set(string.punctuation)
    except_punc = ['?', '!', '\"', ',', '.']
    for ex in except_punc:
        punctuation.remove(ex)
    out = []
#     print(list(text))
#     out = [e for e in list(text) if e not in punctuation]
    for e in list(text):

        if e in except_punc:
            if e != '.' :
                out.append(" "+e)
            else :
                out.append(".")
        elif e not in punctuation or e is '\'':
            out.append(e)
        else :
            out.append(" ")
    return "".join(out)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


# Decontracrion

In [4]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# Split contraction (include 'em 'til)

In [5]:
import re

def split_contraction(phrase):
    # convert two types of single qoute
    phrase = re.sub(r"’", "'", phrase)
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    
    phrase = re.sub(r"\'em ", "them ", phrase)
    phrase = re.sub(r"\'til", "until", phrase)
    
    phrase = re.sub(r"y\'", " you ", phrase)
    
    phrase = re.sub(r"in'", "ing", phrase)
    
    # general

    phrase = re.sub(r"n \'t", " n\'t", phrase)
    
    phrase = re.sub(r"n\'t", " n\'t", phrase)
    phrase = re.sub(r"\'re", " 're", phrase)
    phrase = re.sub(r"\'s", " 's", phrase)
    phrase = re.sub(r"\'d", " 'd", phrase)
    phrase = re.sub(r"\'ll", " 'll", phrase)
    phrase = re.sub(r"\'ve", " 've", phrase)
    phrase = re.sub(r"\'’m", " 'm", phrase)
    
    return phrase

# Group up contigous white space

In [6]:
def group_space(text):
    return " ".join(text.split())

# ' Expand the qoutation mark '

In [7]:
def expand_qoute(text):
    # remove punctuation first
    out = []
    for i in range(len(text)):
        if text[i] == "\'": 
            if(i!=0 and text[i-1] == ' '): #start qoute
                out.append("' ")
            elif(text[i+1] == ' ' and text[i-1]!=" "): #end qoute
                out.append(" '")
        else: out.append(text[i])
    return "".join(out)

# Pad < end >

In [8]:
from nltk.tokenize import sent_tokenize
def find_dot(text):
    index = -1 
    reverse_text = text[::-1]
    for i in range(len(reverse_text)):
        if(reverse_text[i] =='.'):
            index = len(text)-i
        else:
            break
    return index

def pad_end_of_sentence(sentences):
    out = []
    for sentence in sentences:
        if len(sentence) > 0 and sentence[-1] == '.':
            out.append(sentence[:find_dot(sentence)-1])
        else: 
            out.append(sentence)
    return " <end> ".join(out) + ' <end>'
 

# Capital check

In [9]:
from nltk.tokenize import word_tokenize
lowerToCapital = dict()
lowerSet = set()
lowerToCapital['i'] = 'I'
def capital_clean(sentence):
    words = word_tokenize(sentence)
    for word in words[1:]:
        try:
            if word.islower():
                lowerSet.add(word)
            if(word[0].isupper() and word[1].islower()):
                lowerToCapital[word.lower()] = word
        except:
            pass
    return sentence.lower()
    
print(capital_clean("Hey ! ! ! I 've JUST met you Mr. Prayuth."))

print(capital_clean("Hey,  that's... cute... Do you have a nickname, something a little... shorter?"))

print("Captital_Dict:", lowerToCapital)
print("lower set:", lowerSet)

hey ! ! ! i 've just met you mr. prayuth.
hey,  that's... cute... do you have a nickname, something a little... shorter?
Captital_Dict: {'i': 'I', 'mr.': 'Mr.', 'prayuth': 'Prayuth', 'do': 'Do'}
lower set: {'that', 'nickname', 'met', 'a', 'cute', 'little', 'shorter', 'you', "'ve", 'have', "'s", 'something'}


# Check symbols

In [10]:
def have_alphabet(sentence):
    for char in sentence:
        if char.isalpha():
            return True
    return False

In [11]:
from nltk.tokenize import sent_tokenize
def clean_data_main(data):
    out = []
    for line in data:
        if(type(line) is float):
            out.append([" "])
            continue
        sentences = sent_tokenize(line)
        out_sentences = []
        for sentence in sentences :
            if not have_alphabet(sentence):
                continue
            x = capital_clean(sentence)
            x = split_contraction(x)
            x = remove_punc(x)
            x = group_space(x)
            out_sentences.append(x)     
        out.append(pad_end_of_sentence(out_sentences))
    return out

# Read data from files

In [12]:
import glob
# files = glob.glob('./transcripts_clean/*.csv')
files = glob.glob('./cornell.csv')
print(files)

# data = pd.read_csv('./transcripts_clean/0101.html.csv')
# print(data.shape)
# display(data.head())

['./cornell.csv']


In [13]:
%%time
import os
for file in files:
    try:
        data = pd.read_csv(file)
        head, tail = os.path.split(file)
        data['sentence_clean'] = clean_data_main(data['sentence'])
        data['filename'] = tail
        col_to_save = ['name', 'filename', 'sentence_clean']
        data[col_to_save].to_csv("./output/" + tail, index=False)
    except:
        print(file + " ERR")
        pass
    
# !python3 combine_csv.py


CPU times: user 1min 43s, sys: 1 s, total: 1min 44s
Wall time: 1min 49s


In [14]:
print(len(lowerToCapital))
display(lowerToCapital)
display(lowerSet)

17807


{'i': 'I',
 'mr.': 'Mr.',
 'prayuth': 'Prayuth',
 'do': 'Do',
 'scene': 'Scene',
 'korrine': 'Korrine',
 'andrew': 'Andrew',
 'barrett': 'Barrett',
 'french': 'French',
 'cameron': 'Cameron',
 'kat': 'Kat',
 'chat': 'Chat',
 'find': 'Find',
 'wench': 'Wench',
 'date': 'Date',
 'deep': 'Deep',
 'conditioner': 'Conditioner',
 'jared': 'Jared',
 'leto': 'Leto',
 'chastity': 'Chastity',
 'guillermo': 'Guillermo',
 'if': 'If',
 'god': 'God',
 'bonchowski': 'Bonchowski',
 'joey': 'Joey',
 'dorsey': 'Dorsey',
 'so': 'So',
 'bogey': 'Bogey',
 'lowenbrau': 'Lowenbrau',
 'saturday': 'Saturday',
 'sears': 'Sears',
 'queen': 'Queen',
 'harry': 'Harry',
 'prada': 'Prada',
 'hi': 'Hi',
 'daddy': 'Daddy',
 'gigglepuss': 'Gigglepuss',
 'club': 'Club',
 'skunk': 'Skunk',
 'lowenstein': 'Lowenstein',
 'because': 'Because',
 'bitches': 'Bitches',
 'who': 'Who',
 'need': 'Need',
 'prozac': 'Prozac',
 'shakespeare': 'Shakespeare',
 'mandella': 'Mandella',
 'susie': 'Susie',
 'high': 'High',
 'school': 'Sch

{'quarry',
 'non-official',
 'afaid',
 'payer',
 'jay',
 'upmost',
 'brig',
 'scurry',
 'figured',
 'frisked',
 'ravioli',
 'gossip',
 "n'that",
 'm-80s',
 'pigeon',
 'checks',
 'nearest',
 'interfaces',
 'occurrence',
 'narrative',
 'brooded',
 'clover',
 'hobbled',
 'detrimental',
 'pinches',
 'ventriloquism',
 'healthy',
 'consist',
 'tramples',
 'resulting',
 'groop',
 'batsman',
 'ludicrous',
 'methylcyanoacrylate',
 'store',
 'issue',
 'butt-willies',
 'fan',
 'lovesick',
 'shielding',
 'errors',
 'wiping',
 'anti-immigration',
 'mystification',
 'altars',
 'cobwebs',
 'maintained',
 'abou',
 'clue',
 'vastly',
 'radio-controlled',
 'regimes',
 'extractions',
 'stud',
 'skull-',
 'goats',
 'amiable',
 'sendin',
 'employing',
 'palms',
 'spikes',
 'occultism',
 'mournful',
 'draft',
 'subways',
 'pensive',
 'hundred-forty-five',
 'affraid',
 'doings',
 'gradual',
 'starved',
 'heaved',
 'hooked',
 'confrontational',
 'checked',
 'flaxedil',
 'thumbnail',
 "'always",
 'apocalyptic'

In [26]:
for lowerWord in list(lowerSet):
    if(lowerWord in lowerToCapital):
        lowerToCapital.pop(lowerWord)

lowerToCapital['i'] = 'I'
import pickle
f = open("lowerToCappital.pkl","wb")
pickle.dump(lowerToCapital,f)
f.close()

In [27]:
print(len(lowerToCapital))
display(lowerToCapital)

11389


{'mr.': 'Mr.',
 'prayuth': 'Prayuth',
 'korrine': 'Korrine',
 'andrew': 'Andrew',
 'barrett': 'Barrett',
 'cameron': 'Cameron',
 'kat': 'Kat',
 'jared': 'Jared',
 'leto': 'Leto',
 'guillermo': 'Guillermo',
 'bonchowski': 'Bonchowski',
 'joey': 'Joey',
 'dorsey': 'Dorsey',
 'lowenbrau': 'Lowenbrau',
 'saturday': 'Saturday',
 'sears': 'Sears',
 'harry': 'Harry',
 'prada': 'Prada',
 'gigglepuss': 'Gigglepuss',
 'lowenstein': 'Lowenstein',
 'shakespeare': 'Shakespeare',
 'mandella': 'Mandella',
 'susie': 'Susie',
 'bianca': 'Bianca',
 'gloria': 'Gloria',
 'steinem': 'Steinem',
 'brucie': 'Brucie',
 'dakota': 'Dakota',
 'clint': 'Clint',
 'eastwood': 'Eastwood',
 'stratford': 'Stratford',
 'canada': 'Canada',
 'alcatraz': 'Alcatraz',
 'katarina': 'Katarina',
 'thai': 'Thai',
 'kmart': 'Kmart',
 'sparky': 'Sparky',
 'verona': 'Verona',
 'william': 'William',
 'sarah': 'Sarah',
 'lawrence': 'Lawrence',
 'archie': 'Archie',
 'veronica': 'Veronica',
 'wo-man': 'Wo-man',
 'friday': 'Friday',
 'b

# Reverse Function

In [17]:
def recontraction(phrase):
    phrase = re.sub(r"’", "'", phrase)
    # specific
    phrase = re.sub(r"will not", "won\'t", phrase)
    phrase = re.sub(r"can not", "can\'t", phrase)
    
    phrase = re.sub(r"them", "\'em", phrase)
    phrase = re.sub(r"until", "\'til", phrase)
    
    phrase = re.sub(r"you know", "y'know", phrase)
    
    phrase = re.sub(r"ing", "in'", phrase)
    
    # general

    phrase = re.sub(r" n\'t", "n't", phrase)
    phrase = re.sub(r" \'re", "'re", phrase)
    phrase = re.sub(r" \'s", "'s", phrase)
    phrase = re.sub(r" \'d", "'d", phrase)
    phrase = re.sub(r" \'ll", "'ll", phrase)
    phrase = re.sub(r" \'ve", "'ve", phrase)
    phrase = re.sub(r" \'’m", "'m", phrase)
    
    return phrase
    
# test = "Hey I'm Yann, how're you and how's it going ? That's interesting: I'd love to hear more about it."
# test2 = "I don't like a dog"

# out = split_contraction(test)
# print(out)
# print(recontraction(out))

In [18]:
def apply_capital(line):
    out = []
    startWord = True
    for word in line.split():
        if(startWord) :
            out.append(word.capitalize())
            startWord = False
            continue
        if word in lowerToCapital :
            out.append(lowerToCapital[word])
        else :
            out.append(word)
    return " ".join(out)


apply_capital("hey ! ! ! i 've just met you mr. prayuth.")

"Hey ! ! ! i 've just met you Mr. prayuth."

In [19]:
import string
def remove_space_between_punctuation(line) :
    out = ''
    punctuation = set(string.punctuation)
    for i in range(len(line)) :
        try:
            if line[i] == ' ' and line[i-1] in punctuation and line[i+1] in punctuation:
                pass
            else :
                out+=line[i]
        except :
            pass
    return out

remove_space_between_punctuation("! ! ! ! ! Hello@##")
            

'!!!!! Hello@##'

In [20]:
def reverse_clean(data) :
    endding_mark = ['.', '!', '?', ',']
    print(data)
    out = []
    sentences = data.split('<end>')
    for sentence in sentences:
        if len(sentence)<=0:
            continue
        x = recontraction(sentence)
        x = remove_space_between_punctuation(x)
        x = apply_capital(x)
        x = x.strip()
        if(x[-1] not in endding_mark):
            x+='.'    
        print(x)
        out.append(x)
    return " ".join(out)


In [21]:
# reverse_clean("Hey test <end> hello how are you <end>")

In [22]:
reverse_clean("i do n't want to know how to say that though <end> i want to know useful things <end> like where the good stores are <end> how much does champagne cost ? <end> stuff like chat <end> i have never in my life had to point out my head to someone <end>")

i do n't want to know how to say that though <end> i want to know useful things <end> like where the good stores are <end> how much does champagne cost ? <end> stuff like chat <end> i have never in my life had to point out my head to someone <end>
I don't want to know how to say that though.
I want to know useful thin's.
Like where the good stores are.
How much does champagne cost ?
Stuff like chat.
I have never in my life had to point out my head to someone.


"I don't want to know how to say that though. I want to know useful thin's. Like where the good stores are. How much does champagne cost ? Stuff like chat. I have never in my life had to point out my head to someone."

4